# **Compliance GPT with LangChain**

## **Setup**

In [1]:
import nest_asyncio
nest_asyncio.apply()

from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

In [15]:
USER_ID = 'arkan'
CONVERSATION_ID = 'arkan-2021-07-01-01'

## **Define Model**

In [4]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Load Vector Store Index**

In [5]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='bi', embed_model=embed_model, config=config, db_id=0)
vector_store = redis.load_vector_index()

c:\Users\arkan\Downloads\Work\ocbc\ocbc_compliance_gpt\env\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


## **Get Retriever**

In [10]:
from retriever.retriever_bi.retriever_bi import get_retriever_bi

retriever = get_retriever_bi(vector_store=vector_store, top_n=4, top_k=16, llm_model=llm_model, embed_model=embed_model, config=config)

## **Create Chain**

In [11]:
from constant.ojk.prompt import CONTEXTUALIZE_Q_PROMPT_STR, QA_SYSTEM_PROMPT_STR
from databases.chat_store import RedisChatStore
from chain.chain_bi.chain_bi import create_bi_chain
from chain.rag_chain import create_chain_with_chat_history

chat_store = RedisChatStore(k=4, config=config, db_id=1)

chain = create_bi_chain(
    contextualize_q_prompt_str=CONTEXTUALIZE_Q_PROMPT_STR,
    qa_system_prompt_str=QA_SYSTEM_PROMPT_STR,
    retriever=retriever,
    llm_model=llm_model,
    # chat_store=chat_store
)

chain_history = create_chain_with_chat_history(
    final_chain=chain,
    chat_store=chat_store
)

## **Invoke**

#### **With Question, Answer, and Context**

In [12]:
from chain.rag_chain import get_response

response = get_response(
    chain=chain_history,
    question="Apa kepanjangan dari QRIS?",
    user_id=USER_ID,
    conversation_id=CONVERSATION_ID
)

response

{'rewrited question': 'Apa kepanjangan dari QRIS?',
 'answer': 'QRIS merupakan kepanjangan dari Standar Nasional QRCodePembayaran (Quick Response Code Indonesian Standard).\n\nSumber: [Peraturan Anggota Dewan Gubernur Nomor 21/18/PADG/2019](https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf)',
 'context': '[\n  {\n    "metadata": {\n      "id": "doc:bi:e2f84f2b1e474d83a0a7ecc27cec266d",\n      "file_id": "db344534",\n      "title": "PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN.",\n      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf",\n      "date": "21 Agustus 2019",\n      "type_of_regulation": "Peraturan Anggota Dewan Gubernur",\n      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",\n      "standardized_extracted_file_name": "",\n      "standardized_file_name": "padg-21_18_padg_2019-21082019-peraturan_anggota_dewan_gubernur_n

In [13]:
print(response['answer'])

QRIS merupakan kepanjangan dari Standar Nasional QRCodePembayaran (Quick Response Code Indonesian Standard).

Sumber: [Peraturan Anggota Dewan Gubernur Nomor 21/18/PADG/2019](https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf)


In [14]:
print(response["context"])

[
  {
    "metadata": {
      "id": "doc:bi:e2f84f2b1e474d83a0a7ecc27cec266d",
      "file_id": "db344534",
      "title": "PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN.",
      "file_link": "https://www.bi.go.id/id/publikasi/peraturan/Documents/faq_padg_211819.pdf",
      "date": "21 Agustus 2019",
      "type_of_regulation": "Peraturan Anggota Dewan Gubernur",
      "sector": "Sistem Pembayaran Dan Pengelolaan Uang Rupiah",
      "standardized_extracted_file_name": "",
      "standardized_file_name": "padg-21_18_padg_2019-21082019-peraturan_anggota_dewan_gubernur_nomor_21_18_padg_2019_tentang_implementasi_standar_nasional_quick_response_code_untuk_pembayaran",
      "page_number": "1",
      "relevance_score": 0.8493475
    },
    "page_content": "StandarNasional QRCodePembayaran (Quick Response Code Indonesian Standard) yang umumnya disingkat sebagai QRIS adalah Standar QRCodePembayaran yang ditetap

#### **Streaming**

In [16]:
from chain.rag_chain import print_answer_stream

print_answer_stream(chain=chain_history, question="Berapa maksimum transaksi QRIS?", user_id=USER_ID, conversation_id=CONVERSATION_ID)

Error in RootListenersTracer.on_chain_end callback: KeyError('answer')
Error in callback coroutine: KeyError('answer')


Jumlah maksimum transaksi yang dapat dilakukan dengan QRIS adalah Rp5.000.000,00 (lima juta rupiah) per transaksi.

Source: PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 23/8/PADG/2021 TENTANG PERUBAHAN ATAS PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN
[Link to the regulation](https://www.bi.go.id/id/publikasi/peraturan/Documents/FAQ_PADG_230821.pdf#page=2)

#### **Chat Store Monitor**

In [17]:
# to print chat history
print(chat_store.get_session_history(user_id=USER_ID, conversation_id=CONVERSATION_ID))

Human: Berapa maksimum transaksi QRIS?
AI: Jumlah maksimum transaksi yang dapat dilakukan dengan QRIS adalah Rp5.000.000,00 (lima juta rupiah) per transaksi.

Source: PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 23/8/PADG/2021 TENTANG PERUBAHAN ATAS PERATURAN ANGGOTA DEWAN GUBERNUR NOMOR 21/18/PADG/2019 TENTANG IMPLEMENTASI STANDAR NASIONAL QUICK RESPONSE CODE UNTUK PEMBAYARAN
[Link to the regulation](https://www.bi.go.id/id/publikasi/peraturan/Documents/FAQ_PADG_230821.pdf#page=2)


In [31]:
# to make the chat history empty
chat_store.clear_all()